# Flow and task configuration

## Basic flow config
Basic flow configuration includes the ability to provide a name, description, and version and other options for the flow via flow arguments.

A flow description enables you to provide documentation right alongside your flow object.

A flow version enables you to associate a given run of your workflow with the version of code or configuration that was used.

In [1]:
from prefect import flow, task


In [ ]:

@flow(name="My Example Flow", 
      description="An example flow for a tutorial.",
      version="tutorial_02")
def my_flow():
    # run tasks and subflows


## Basic task configuration

you can independently assign tasks their own name and description.

In [2]:
@task(name="My example task",
description= "An example task for a tutorial")
def print_hello():
    print("Hello Thanh Tu")

@flow(name="My Example Flow", 
      description="An example flow for a tutorial.",
      version="tutorial_02")
def make_flow():
    print_hello()

make_flow()

21:55:04.283 | INFO    | prefect.engine - Created flow run 'attractive-stoat' for flow 'My Example Flow'
21:55:04.399 | INFO    | Flow run 'attractive-stoat' - Created task run 'My example task-1fa5169b-0' for task 'My example task'
21:55:04.400 | INFO    | Flow run 'attractive-stoat' - Executing 'My example task-1fa5169b-0' immediately...
21:55:04.435 | INFO    | Task run 'My example task-1fa5169b-0' - Finished in state Completed()
21:55:04.449 | INFO    | Flow run 'attractive-stoat' - Finished in state Completed('All states completed.')


Hello Thanh Tu


[Completed(message=None, type=COMPLETED, result=LiteralResult(type='literal', value=None))]

## Flow and task retries
This enables flows and tasks to automatically retry on failure

In [ ]:
@task(retries=2, retry_delay_seconds = 5)
def failure():
    print('running')
    raise ValueError("bad code")

@flow 
def test_retries():
    return failure()



In [4]:
state = test_retries()

22:02:54.941 | INFO    | prefect.engine - Created flow run 'satisfied-caiman' for flow 'test-retries'
22:02:55.037 | INFO    | Flow run 'satisfied-caiman' - Created task run 'failure-f8b6b21b-0' for task 'failure'
22:02:55.038 | INFO    | Flow run 'satisfied-caiman' - Executing 'failure-f8b6b21b-0' immediately...
22:02:55.058 | ERROR   | Task run 'failure-f8b6b21b-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/prefect/engine.py", line 1190, in orchestrate_task_run
    result = await run_sync(task.fn, *args, **kwargs)
  File "/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 68, in run_sync_in_worker_thread
    return await anyio.to_thread.run_sync(call, cancellable=True)
  File "/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_

running


22:02:59.103 | ERROR   | Task run 'failure-f8b6b21b-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/prefect/engine.py", line 1190, in orchestrate_task_run
    result = await run_sync(task.fn, *args, **kwargs)
  File "/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 68, in run_sync_in_worker_thread
    return await anyio.to_thread.run_sync(call, cancellable=True)
  File "/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/anyio/_backends/_async

running


22:03:03.147 | ERROR   | Task run 'failure-f8b6b21b-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/prefect/engine.py", line 1190, in orchestrate_task_run
    result = await run_sync(task.fn, *args, **kwargs)
  File "/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 68, in run_sync_in_worker_thread
    return await anyio.to_thread.run_sync(call, cancellable=True)
  File "/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/anyio/_backends/_async

running


ValueError: bad code

## Task caching
Caching refers to the ability of a task run to reflect a finished state without actually running the code that defines the task.

This allows you to efficiently reuse results of tasks that may be particularly "expensive" to run with every flow run. 

Moreover, Prefect makes it easy to share these states across flows and flow runs using the concept of a "cache key function".

In [ ]:
from datetime import timedelta
from prefect import flow, task
from prefect.tasks import task_input_hash

@task(cache_key_fn=task_input_hash, cache_expiration=timedelta(days=1))
def hello_task(name_input):
    # Doing some work
    print("Saying hello")
    return "hello " + name_input

@flow
def hello_flow(name_input):
    hello_task(name_input)
